# Setup

In [108]:
# OctoAI
# ! pip install langchain langchain-community faiss-cpu sentence-transformers octoai-sdk langchain-text-splitters lxml tiktoken python-dotenv 'arize-phoenix[evals]'

In [109]:
from dotenv import load_dotenv
import os

load_dotenv()
OCTOAI_API_TOKEN = "eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCIsImtpZCI6IjNkMjMzOTQ5In0.eyJzdWIiOiJkNTY1Y2Q3YS0zYmNjLTQzNDgtOGQxYy1mMGY0ZjY0ODkyYzciLCJ0eXBlIjoidXNlckFjY2Vzc1Rva2VuIiwidGVuYW50SWQiOiJkMzRmOGVkZC1kMzE1LTQ4NTktOTc0Zi03MjJiMzNlNDA5ZDIiLCJ1c2VySWQiOiJhZTMxM2ExZS1lYTI3LTRkOTItYjk5OS1iOTNlY2Q0YjQ3NTgiLCJhcHBsaWNhdGlvbklkIjoiYTkyNmZlYmQtMjFlYS00ODdiLTg1ZjUtMzQ5NDA5N2VjODMzIiwicm9sZXMiOlsiRkVUQ0gtUk9MRVMtQlktQVBJIl0sInBlcm1pc3Npb25zIjpbIkZFVENILVBFUk1JU1NJT05TLUJZLUFQSSJdLCJhdWQiOiIzZDIzMzk0OS1hMmZiLTRhYjAtYjdlYy00NmY2MjU1YzUxMGUiLCJpc3MiOiJodHRwczovL2lkZW50aXR5Lm9jdG8uYWkiLCJpYXQiOjE3MTk2Nzg3NTl9.m3RpRr3vLoGekWhvk1fXHlTBAg1fkkO2V2eMfXddjLf30taFXnAOLoJIWHsGLupliLVW3JL9_wxPRhL2cGpDH_1_PeWDUYxKk_ZY1RjL9s_yoJf7fGeEy9-VPbmvjfYXcQTa_sxE99GeqwaODud26e_O04Q9n2KiNSwdeeion6ki9ctpIbDXZ3Wit_ltumGY0axDRInfm_QHwvo92r4W-3g8jN4lskS7ZA7CT5LCRq7GzFs5E2HsZq-59RnVhxnray6nG5fyG3U67Ff0ZCTuRaDY2DqRDOgQw1zRqTYMoW8zeOfozRxlGygdRcRU3AxnUOGOi7yijetkE9I60H9qlg"

# Ingest Data

In [110]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document

In [111]:
files = os.listdir("../workout_data")
file_texts = []
for file in files:
    with open(f"../workout_data/{file}") as f:
        file_text = f.read()
    text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=512, chunk_overlap=64, 
    )
    texts = text_splitter.split_text(file_text)
    for i, chunked_text in enumerate(texts):
        file_texts.append(Document(page_content=chunked_text, 
                metadata={"doc_title": file.split(".")[0], "chunk_num": i}))

Created a chunk of size 626, which is longer than the specified 512
Created a chunk of size 676, which is longer than the specified 512


In [112]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

In [113]:
embeddings = HuggingFaceEmbeddings()

In [114]:
vector_store = FAISS.from_documents(
    file_texts,
    embedding=embeddings
)

# Search the Data

In [115]:
from langchain_community.llms.octoai_endpoint import OctoAIEndpoint
llm = OctoAIEndpoint(
        model="mixtral-8x7b-instruct",
        max_tokens=4096,
        presence_penalty=0,
        temperature=0,
        top_p=0.9,
        octoai_api_token=OCTOAI_API_TOKEN
    )

/home/cmagorian/Storage/repos/hackathon-0629/.venv/lib/python3.10/site-packages/langchain_core/utils/utils.py:161: UserWarning: WARNING! model is not default parameter.
                model was transferred to model_kwargs.
                Please confirm that model is what you intended.
  warnings.warn(


In [116]:
retriever = vector_store.as_retriever()

In [117]:
from langchain.prompts import ChatPromptTemplate
template="""You are a data analyst. You will be given unstructured data texts and figure out the schema. Return the schema. Be as general as possible so the schema can be used on most data input. Return the schema as PostgreSQL CREATE TABLE query. If possible, create multiple tables and relations between them when you see fit.
Document: {question} 
Context: {context} 
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

In [118]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [119]:
print(chain.invoke("Show me the schema creation script"))

 
-- Table: equipment
equipment_id SERIAL PRIMARY KEY,
equipment_name VARCHAR(255) NOT NULL;

-- Table: exercise
exercise_id SERIAL PRIMARY KEY,
exercise_name VARCHAR(255) NOT NULL,
equipment_id INTEGER REFERENCES equipment(equipment_id);

-- Table: workout
workout_id SERIAL PRIMARY KEY,
workout_name VARCHAR(255) NOT NULL;

-- Table: workout_exercise
workout_exercise_id SERIAL PRIMARY KEY,
workout_id INTEGER REFERENCES workout(workout_id),
exercise_id INTEGER REFERENCES exercise(exercise_id),
sets INTEGER NOT NULL,
reps INTEGER NOT NULL;

-- Table: day
day_id SERIAL PRIMARY KEY,
day_number INTEGER NOT NULL,
workout_id INTEGER REFERENCES workout(workout_id);

INSERT INTO equipment (equipment_name) VALUES ('gym'), ('dumbbells'), ('barbell'), ('cable'), ('machine');

INSERT INTO exercise (exercise_name, equipment_id) VALUES 
('barbell back squats', 3),
('flat barbell bench press', 2),
('seated cable rows', 4),
('seated dumbbell shoulder press', 2),
('cable rope triceps pushdowns', 4),
('l